# Final Project - Part 4 - Model Draft 1 (Demo vs Exploit Geo) - 0.87-0.88

## 1) Multinomial Logistic Regression (Input: Demographics; Output: Exploitation Geo Category)

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import cross_val_score,train_test_split
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.base import BaseEstimator
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import random
import matplotlib as plt
import pdb

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import cross_val_score,train_test_split
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.base import BaseEstimator
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import random
import matplotlib as plt
import pdb

C:\Users\ellei\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
#Read Cleaned Global Data
GD_data_final = pd.read_csv('assets/GD_data_final.csv')
GD_data_final.head()

,yearOfRegistration,gender,majorityStatus,ageBroad_mid,citizenshipRegion,citizenshipGeoCategory,exploitationRegion,exploitationGeoCategory,mocDebtBondage,mocTakesEarnings,...,tosProstitution,tosPornography,tosRemoteInteractiveServices,tosPrivateSexualServices,tosNotSpecified,rrIntimatePartner,rrFriend,rrFamily,rrOther,rrUnknown
0,2010,Female,Adult,19.0,Central_Asia,Asia,Central_Asia,Asia,0,0,...,0,0,0,0,1,0,0,0,0,1
1,2004,Female,Adult,19.0,Eastern_Europe,Europe,Eastern_Europe,Europe,0,0,...,0,0,0,0,1,0,0,0,0,1
2,2010,Female,Adult,19.0,Central_Asia,Asia,Central_Asia,Asia,1,0,...,0,0,0,0,1,0,0,0,0,1
3,2010,Female,Adult,19.0,Central_Asia,Asia,Central_Asia,Asia,1,0,...,0,0,0,0,1,0,0,0,0,1
4,2012,Female,Minor,13.0,North_America,North_America,North_America,North_America,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
# Read Demographic vs Exploitation Geo Data
demo_expgeo_calc = pd.read_csv('assets/demo_expgeo_calc.csv')
demo_expgeo_calc2 = demo_expgeo_calc.iloc[:,:25].join(GD_data_final[['exploitationRegion','exploitationGeoCategory']])
demo_expgeo_calc2.columns

Index([u'yearOfRegistration', u'ageBroad_mid', u'g_Female', u'g_Male',
       u'ms_Adult', u'cr_Central_Asia', u'cr_East_Africa', u'cr_East_Asia',
       u'cr_Eastern_Europe', u'cr_Middle_East', u'cr_North_America',
       u'cr_Northeast_Africa', u'cr_South_America', u'cr_South_Asia',
       u'cr_Southeast_Asia', u'cr_Southeastern_Europe', u'cr_Unknown',
       u'cr_West_Africa', u'cg_Africa', u'cg_Asia', u'cg_Europe',
       u'cg_Middle_East', u'cg_North_America', u'cg_South_America',
       u'cg_Unknown', u'exploitationRegion', u'exploitationGeoCategory'],
      dtype='object')

In [6]:
len(demo_expgeo_calc2['cg_South_America'])

18484

In [15]:
# Multinomial Logistic Regression
def demo_expgeo_main():
    ### Including all demo features (Can be modified and observe score changes):
    #demo_expgeo_headers = ['yearOfRegistration', 'ageBroad_mid', 'g_Female', 'g_Male', 'ms_Adult', 
                              #'cr_Central_Asia', 'cr_East_Africa', 'cr_East_Asia', 'cr_Eastern_Europe', 'cr_Middle_East', 
                              #'cr_North_America', 'cr_Northeast_Africa', 'cr_South_America', 'cr_South_Asia', 
                              #'cr_Southeast_Asia', 'cr_Southeastern_Europe', 'cr_Unknown', 'cr_West_Africa', 
                              #'cg_Africa', 'cg_Asia', 'cg_Europe', 'cg_Middle_East', 'cg_North_America', 'cg_South_America', 
                              #'cg_Unknown', 'exploitationRegion', 'exploitationGeoCategory']
    demo_expgeo_headers = ['yearOfRegistration','ageBroad_mid', 'g_Female', 'ms_Adult', 
                              'cr_Central_Asia', 'cr_East_Africa', 'cr_East_Asia', 'cr_Eastern_Europe', 'cr_Middle_East', 
                              'cr_North_America', 'cr_Northeast_Africa', 'cr_South_America', 'cr_South_Asia', 
                              'cr_Southeast_Asia', 'cr_Southeastern_Europe', 'cr_West_Africa', 
                              'cg_Africa', 'cg_Asia', 'cg_Europe', 'cg_Middle_East', 'cg_North_America', 'cg_South_America','exploitationRegion', 'exploitationGeoCategory']
    
    ### Train Test Split - Out put col index -1 is Geo Category, -2 is Region
    X = demo_expgeo_calc2[demo_expgeo_headers[:-2]]
    y = demo_expgeo_calc2[demo_expgeo_headers[-1]]
    train_x, test_x, train_y, test_y = train_test_split(X, y, train_size=0.7)
    
    # USE TWO DIFFERENT APPROACHES TO TRAIN 
    # Train multi-classification model with logistic regression
    lr = LogisticRegression()
    lr.fit(train_x, train_y)
 
    # Train multinomial logistic regression model
    mul_lr = LogisticRegression(multi_class='multinomial', solver='newton-cg')
    mul_lr.fit(train_x, train_y)
 
    print "Logistic regression Train Accuracy :: ", metrics.accuracy_score(train_y, lr.predict(train_x))
    print "Logistic regression Test Accuracy :: ", metrics.accuracy_score(test_y, lr.predict(test_x))
 
    print "Multinomial Logistic regression Train Accuracy :: ", metrics.accuracy_score(train_y, mul_lr.predict(train_x))
    print "Multinomial Logistic regression Test Accuracy :: ", metrics.accuracy_score(test_y, mul_lr.predict(test_x))
    
    print ("Cross validating...")
    model_demo_expgeo = LogisticRegression(multi_class='multinomial', solver='newton-cg')
    ### Only accuracy score works because it's multinomial regression
    print ("Multi-classification Log Reg - train accuracy: ")
    print (np.mean(cross_val_score(lr, train_x, train_y.values, scoring='accuracy', cv=10)))
    print ("Multinomial Log Reg - train accuracy: ")
    print (np.mean(cross_val_score(mul_lr, train_x, train_y.values, scoring='accuracy', cv=10)))
    #print ("precision: ")
    #print (np.mean(cross_val_score(model_demo_expgeo, train_x, train_y.values, scoring='precision', cv=10)))
    #print ("recall: ")
    #print (np.mean(cross_val_score(model_demo_expgeo, train_x, train_y.values, scoring='recall', cv=10)))
    #print ("f1: ")
    #print (np.mean(cross_val_score(model_demo_expgeo, train_x, train_y.values, scoring='f1', cv=10)))
    #print ("roc_auc: ")
    #print (np.mean(cross_val_score(model_demo_expgeo, train_x, train_y.values, scoring='roc_auc', cv=10)))

if __name__ == '__main__':
  demo_expgeo_main()


Logistic regression Train Accuracy ::  0.884603493585
Logistic regression Test Accuracy ::  0.875946628201
Multinomial Logistic regression Train Accuracy ::  0.884526201886
Multinomial Logistic regression Test Accuracy ::  0.873422286332
Cross validating...
Multi-classification Log Reg - train accuracy: 
0.879889250394
Multinomial Log Reg - train accuracy: 
0.883908409522


### Findings:
Cross Val Score - Accuracy: (Generally between 0.87-0.88)
- Include all features = 0.879
- All except yearOfRegistration = 0.874
- All except yearOfRegistration, g_Male, cr_Unknown, cg_Unknown = 0.878 (remove possibly multicollinearity)

### Accuracy Scores by Features - Output - exploitationGeoCategory:
#### yearOfRegistration
Multinomial Logistic regression Train Accuracy ::  0.603802751584
Multinomial Logistic regression Test Accuracy ::  0.597728092319

#### ageBroad_Mid
Multinomial Logistic regression Train Accuracy ::  0.555881898284
Multinomial Logistic regression Test Accuracy ::  0.545798773891

#### g_Female
Multinomial Logistic regression Train Accuracy ::  0.578219199258
Multinomial Logistic regression Test Accuracy ::  0.580057699243

#### ms_Adult
Multinomial Logistic regression Train Accuracy ::  0.513294172206
Multinomial Logistic regression Test Accuracy ::  0.513342949874

#### citizenship_Region
Multinomial Logistic regression Train Accuracy ::  0.849203895502
Multinomial Logistic regression Test Accuracy ::  0.845293905517

#### citizenship_Geo_Category
Multinomial Logistic regression Train Accuracy ::  0.802597001082
Multinomial Logistic regression Test Accuracy ::  0.803281644428

#### All Demo Features:
Multinomial Logistic regression Train Accuracy ::  0.880584325243
Multinomial Logistic regression Test Accuracy ::  0.879733141003

### Accuracy Scores by Features - Output - exploitationRegion:
#### All Demo Features:
Multinomial Logistic regression Train Accuracy ::  0.880584325243
Multinomial Logistic regression Test Accuracy ::  0.879733141003